## Clustering Counties Within a State Based on Industry Characteristics

Four WD types:

- Residential
- Highway
- Building
- Heavy

QCEW provides the following useful characteristics by industry and county (this is not an exhaustive list):

- Average annual number of establishments
- Average annual employment
- Average weekly wage
- Annual contributions 

We can map NAICS to WD type as follows:

- **Residential:** NAICS 2361 Residential Building Construction
- **Highway:** NAICS 2373 Highway, Street, and Bridge Construction
- **Building:** NAICS 2362 Nonresidential Building Construction
- **Heavy:** NAICS 2379 Other Heavy Construction

With this simple mapping, counties can easily be compared to one another based on industry characteristics. 

In [1]:
using CSV
using Clustering
using DataFrames
using DataFramesMeta
using Distances
using Interact
using StatsBase
using URIParser
using VegaLite
using WebIO;

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-17343010142586517998\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-13200645692505111893\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-17343010142586517998\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

#### Create a dictionary mapping NAICS codes to QCEW file names

In [2]:
const industries = Dict(
    2361 => "2018.annual 2361 NAICS 2361 Residential building construction.csv",
    2373 => "2018.annual 2373 NAICS 2373 Highway, street, and bridge construction.csv",
    2362 => "2018.annual 2362 NAICS 2362 Nonresidential building construction.csv",
    2379 => "2018.annual 2379 NAICS 2379 Other heavy construction.csv"
);

#### Load the state abbreviations/fips file

In [3]:
const states_abbrevs_fips = @linq DataFrame(CSV.read("data/states_abbrebs_fips.csv")) |>
    transform(fips = @. lpad(string(:fips), 2, "0"))

,state,abbrev,fips
,String,String,String
1,alabama,AL,01
2,alaska,AK,02
3,arizona,AZ,04
4,arkansas,AR,05
5,california,CA,06
6,colorado,CO,08
7,connecticut,CT,09
8,delaware,DE,10
9,district-of-columbia,DC,11


#### Write a function that returns a `DataFrame` for a particular industry and state, filtered

In [4]:
function industry_df(state::String, industry::Int)
    @linq DataFrame(CSV.File("data/$(industries[industry])", normalizenames=true)) |>
        where(:agglvl_title .== "County, NAICS 4-digit -- by ownership sector") |>
        where(:own_title .== "Private") |>
        where(first.(:area_fips, 2) .== state)
end;

#### Write a function that converts a `DataFrame` to a normalized `Matrix`

In [5]:
function normalize_matrix(df::DataFrame)
   matrix = Matrix(hcat(
        df.annual_avg_estabs_count,
        df.annual_avg_emplvl,
        df.annual_avg_wkly_wage,
        df.annual_contributions
    )')
    return StatsBase.transform(fit(ZScoreTransform, matrix, dims=2), convert.(Float64, matrix)) 
end;

In [6]:
function distance_matrix(state::String, industry::Int, distance_metric::PreMetric)
    df = industry_df(state, industry)
    matrix = normalize_matrix(df)
    dm = (df.area_title, Distances.pairwise(distance_metric, matrix, dims=2))
    distance_df =  DataFrame(dm[2])
    rename!(distance_df, Symbol.(dm[1]))
    insertcols!(distance_df, 1, :County => dm[1])
    return distance_df
end;

#### Write a function that returns an `Expression` to generate the interactive plot

In [7]:
function generate_plot(link, object_name, groups)
   return :(
        @vlplot(width=1200, height=900) + 
        @vlplot(
            mark={ 
                :geoshape,
                stroke=:black
            },
            data={
                url=URI($link),
                format={
                    type=:topojson,
                    feature=$object_name
                }
            },
            transform=[
                {
                    lookup="properties.GEOID",
                    from={
                        data=$groups,
                        key=:fips,
                        fields=["group"]
                    }
                }
            ],
            color={
                "group:n",
                legend={title="Group"}
            },
            projection={
                typ=:naturalEarth1
            }
        ) +
        @vlplot(
            :text,
            data={
                url=URI($link),
                format={
                    type=:topojson,
                    feature=$object_name
                }
            },
                transform=[
                    {
                        calculate="geoCentroid(null, datum)",
                        as="centroid"
                    },
                    {
                        calculate="datum.centroid[0]",
                        as="centroidx"
                    },
                    {
                        calculate="datum.centroid[1]",
                        as="centroidy"
                    }
                ],
            text={field="properties.NAME", type=:nominal},
            longitude="centroidx:q",
            latitude="centroidy:q"
        )
    )
end;

### Top Down vs. Bottom Up

We can take a top-down approach by determining how many groups we want for a given state, and then clustering the counties accordingly. Conversely, we can simply compute a distance matrix for each state which would allow the grouping of counties in bottom-up fashion, only when necessary.

Below is a distance matrix for the state of Connecticut:

In [8]:
ENV["COLUMNS"]=120 # Make Jupyter show more columns
distance_matrix("10", 2361, Euclidean())[1:3,1:4]

,County,"Kent County, Delaware","New Castle County, Delaware","Sussex County, Delaware"
,String,Float64,Float64,Float64
1,"Kent County, Delaware",0.0,1.65955,3.65433
2,"New Castle County, Delaware",1.65955,0.0,3.49941
3,"Sussex County, Delaware",3.65433,3.49941,0.0


### Fuzzy C Means

In [22]:
function create_groups_fuzzy_cmeans(df::DataFrame, matrix::Matrix; C::Int=4, m::Float64=2.75)
    weights = fuzzy_cmeans(matrix, C, m).weights
    df = DataFrame(
        fips = df.area_fips,
        county = df.area_title,
        group = [findfirst(x -> x == maximum(weights[i,:]), weights[i,:]) for i = 1:size(weights,1)]
    )
    return df
end;

In [26]:
function show_state_groups_fuzzyc(state::String, industry::Int, C::Int=4, m::Float64=2.75)
    link = "https://raw.githubusercontent.com/mthelm85/topojson/master/countries/us-states/$(states_abbrevs_fips[states_abbrevs_fips.fips .== state, :abbrev][1])-$(state)-$(states_abbrevs_fips[states_abbrevs_fips.fips .== state, :state][1])-counties.json"
    object_name = replace("cb_2015_$(states_abbrevs_fips[states_abbrevs_fips.fips .== state, :state][1])_county_20m", "-" => "_")
    return @manipulate for C = slider(2:20, value=C, label="Number of Groups"), m = slider(1.1:0.1:10.0, value=2.75, label="Fuzziness Factor")
        df = industry_df(state, industry)
        matrix = normalize_matrix(df)
        groups = create_groups_fuzzy_cmeans(df, matrix; C=C, m=m)
        eval(generate_plot(link, object_name, groups))
    end
end;

In [27]:
show_state_groups_fuzzyc("20", 2361)

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["Number of Groups"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 20,:min => 2,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Int64} with 2 listeners. Value:
4, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "C:\\Users\\mthel\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\mthel\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\mthel\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\mthel\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\mthel\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000015f508b0, Task (runnable) @0x0000000015f508b0), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"2\",\"3\",\"4\",\"5\",\"6\",\"7\",\"8\",\"9\",\"10\",\"11\",\"12\",\"13\",\"14\",\"15\",\"16\",\"17\",\"18\",\"19\",\"20\"],\"changes\":WebIO.getval({\"name\":\"changes\",\

### K Medoids

In [12]:
function create_groups_kmedoids(df::DataFrame, matrix::Matrix, k::Int=4)
    assignments = kmedoids(Distances.pairwise(Euclidean(), matrix, dims=2), k).assignments
    df = DataFrame(
        fips = df.area_fips,
        county = df.area_title,
        group = assignments
    )
    return df
end;

In [13]:
function show_state_groups_kmeds(state::String, industry::Int, k::Int=4)
    link = "https://raw.githubusercontent.com/mthelm85/topojson/master/countries/us-states/$(states_abbrevs_fips[states_abbrevs_fips.fips .== state, :abbrev][1])-$(state)-$(states_abbrevs_fips[states_abbrevs_fips.fips .== state, :state][1])-counties.json"
    object_name = replace("cb_2015_$(states_abbrevs_fips[states_abbrevs_fips.fips .== state, :state][1])_county_20m", "-" => "_")
    return @manipulate for k = slider(2:12, value=k, label="Number of Groups")
        df = industry_df(state, industry)
        matrix = normalize_matrix(df)
        groups = create_groups_kmedoids(df, matrix, k)
        eval(generate_plot(link, object_name, groups))
    end
end;

In [14]:
show_state_groups_kmeds("10", 2361)

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["Number of Groups"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 12,:min => 2,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Int64} with 2 listeners. Value:
4, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "C:\\Users\\mthel\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\mthel\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\mthel\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\mthel\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\mthel\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000031f54010, Task (runnable) @0x0000000031f54010), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"2\",\"3\",\"4\",\"5\",\"6\",\"7\",\"8\",\"9\",\"10\",\"11\",\"12\"],\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"9749135233383646271\",\"id\":\"15221639316876

### DBSCAN

In [32]:
function create_groups_dbscan(df::DataFrame, matrix::Matrix, ϵ::Float64=0.3, min_points::Int=2)
    assignments = dbscan(Distances.pairwise(Euclidean(), matrix, dims=2), ϵ, min_points).assignments
    df = DataFrame(
        fips = df.area_fips,
        county = df.area_title,
        group = assignments
    )
    return df
end;

In [33]:
function show_state_groups_dbscan(state::String, industry::Int, ϵ::Float64=0.3, min_points::Int=2)
    link = "https://raw.githubusercontent.com/mthelm85/topojson/master/countries/us-states/$(states_abbrevs_fips[states_abbrevs_fips.fips .== state, :abbrev][1])-$(state)-$(states_abbrevs_fips[states_abbrevs_fips.fips .== state, :state][1])-counties.json"
    object_name = replace("cb_2015_$(states_abbrevs_fips[states_abbrevs_fips.fips .== state, :state][1])_county_20m", "-" => "_")
    return @manipulate for ϵ = slider(0.1:0.1:2.0, value=ϵ, label="Radius"), min_points = slider(2:8, value=min_points, label="Min. Points")
        df = industry_df(state, industry)
        matrix = normalize_matrix(df)
        groups = create_groups_dbscan(df, matrix, ϵ, min_points)
        eval(generate_plot(link, object_name, groups))
    end
end;

In [37]:
show_state_groups_dbscan("10", 2361)

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["Radius"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 20,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
3, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "C:\\Users\\mthel\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\mthel\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\mthel\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\mthel\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\mthel\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000015ce6570, Task (runnable) @0x0000000015ce6570), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.1\",\"0.2\",\"0.3\",\"0.4\",\"0.5\",\"0.6\",\"0.7\",\"0.8\",\"0.9\",\"1.0\",\"1.1\",\"1.2\",\"1.3\",\"1.4\",\"1.5\",\"1.6\",\"1.7\",\"1.8\",\"1.9\",\"2.0\"],\"changes\":WebIO.getval(

### How does this work?

We can compare counties by storing the relevant characteristics inside of a vector. For example, suppose we have the following:

- **County 1:**
    - 20 average establishments
    - 250 average employees
    - 300 average weekly wage
    - 12,000 average contributions
  
- **County 2:**
    - 30 average establishments
    - 300 average employees
    - 270 average weekly wage
    - 10,000 average contributions
    
- **County 3:**
    - 50 average establishments
    - 900 average employees
    - 560 average weekly wage
    - 19,000 average contributions
    
In this case, each county can be represented as follows:

\$c_1$ = $[20,250,300,12000]$

\$c_2$ = $[30,300,270,10000]$

\$c_3$ = $[50,900,560,19000]$

Now, using linear algebra, we can compute the Euclidean (straight-line) distance between each of these vectors, just as we would compute the distance between two points on a two-dimensional $xy$ plane. The Euclidean distance between two vectors $a$ and $b$ is defined as the magnitude (norm) of their difference:

$dist(a,b) = ||a - b||$

The formula to compute the norm of a vector $c$, of length $n$, is $\sqrt{c_{1}^2 + c_{2}^2 + ... + c_{n}^2}$. Therefore, the Euclidean distance between two vectors $a$ and $b$ is simply $\sqrt{(a_{1}-b_{1})^2 + (a_{2}-b_{2})^2 + ... + (a_{n}-b_{n})^2}$.

### Computing Euclidean distances with Julia

In [18]:
c₁ = [20,250,300,12000]
c₂ = [30,300,270,10000]
c₃ = [50,900,560,19000];

### Normalizing the data

We must normalize our vectors since the units of measurement are different for the different characteristics:

In [19]:
c₁ = StatsBase.transform(fit(ZScoreTransform, c₁, dims=1), convert.(Float64, c₁))
c₂ = StatsBase.transform(fit(ZScoreTransform, c₂, dims=1), convert.(Float64, c₂))
c₃ = StatsBase.transform(fit(ZScoreTransform, c₃, dims=1), convert.(Float64, c₃));

### Computing the distance manually

We can compute the distance between $c_1$ and $c_2$ manually by simply subtracting $c_2$ from $c_1$ in element-wise fashion, squaring each of the results, taking their sum, and then taking the square root of that:

In [20]:
sqrt(sum(@. (c₁-c₂)^2))

0.012594584572059711

We can also utilize the ```norm``` function from the ```LinearAlgebra``` package:

In [21]:
using LinearAlgebra

In [22]:
norm(c₁-c₂)

0.012594584572059711

In [23]:
norm(c₁-c₃)

0.0404282966408695

In [24]:
norm(c₂ -c₃)

0.02785508802843638

### Interpreting the results

We can see from this comparison that counties $c_1$ and $c_2$ are the most similar because the Euclidean distance is the smallest. $c_1$ and $c_3$ are the most different. $c_2$ and $c_3$ are more similar than $c_1$ and $c_3$, but more different than $c_1$ and $c_2$. We can verify that the ```Distances``` package we are using arrives at the same conclusions:

In [25]:
Euclidean()(c₁,c₂)

0.012594584572059711

In [26]:
Euclidean()(c₁,c₃)

0.0404282966408695

In [27]:
Euclidean()(c₂,c₃)

0.02785508802843638

### Clustering

Once we have computed the distance between every county in a state and every other county in that same state, we can then apply a variety of clustering algorithms to achieve the groupings. Each algorithm works differently but they all group counties based on the Euclidean distances from one another, computed above.